In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


In [5]:

# Load the HTML content from the uploaded file
with open('..\web_registeration\Search for Courses_.html', 'r') as file:
    html_content = file.read()

# Parse the HTML content using Beautiful Soup
soup = BeautifulSoup(html_content, 'html.parser')

In [13]:
# Initialize a list to hold all course information
all_courses = []

# Find all program names
programs = soup.find_all(['td'], class_='deheader')
program_names = []
for program in programs:
    program_name = program.find('span', class_='LargerText').text.strip()
    program_names.append(program_name)


In [16]:
# get column names
column_names_tags = soup.find('tr', bgcolor="#FFFFCC").find_all('b')
column_names = [tag.get_text(strip=True) for tag in column_names_tags]

In [61]:
# Step 1: Split the HTML by Programs
program_blocks = soup.find_all('td', class_='deheader')

# Prepare a list to hold all courses across programs
all_courses = []
expected_details_length = 19  # Set this to the expected number of <td> elements in informative rows

for program_block in program_blocks:
    program_name = program_block.find('span', class_='LargerText').get_text(strip=True)
    program_row = program_block.find_parent('tr')
    next_sib = program_row.find_next_sibling('tr')

    while next_sib and not next_sib.find('td', class_='deheader'):
        course_code_tag = next_sib.find('td', colspan="19", class_='dedefault')
        if course_code_tag:
            course_code = course_code_tag.get_text(strip=True)
            next_sib = next_sib.find_next_sibling('tr')

        while next_sib and not next_sib.find('td', colspan="19", class_='dedefault') and not next_sib.find('td', class_='deheader'):
            details = [td.get_text(strip=True) for td in next_sib.select('td.dedefault, td.dedead')]
            
            # Only process rows where the length of details matches the expected length
            if len(details) == expected_details_length:
                course_details = {
                    'Program Name': program_name,
                    'Course Code': course_code,
                    # Map the extracted details to the respective fields
                    'CRN': details[4],
                    'Subject': details[5],
                    'Course Number': details[6],
                    'Section': details[7],
                    'Title': details[9],
                    'Type': details[12],
                    'Day(s)': details[13],
                    'Time': details[14],
                    'Room': details[17],
                    'Instructor(s)': details[18]
                }
                all_courses.append(course_details)

            next_sib = next_sib.find_next_sibling('tr')
# Print or process the extracted courses
for course in all_courses:
    print(course)

{'Program Name': 'Anthropology', 'Course Code': 'ANTH 1120', 'CRN': '20290', 'Subject': 'ANTH', 'Course Number': '1120', 'Section': '001', 'Title': 'Intro\xa0to\xa0Cultural Anthropology', 'Type': 'Lecture', 'Day(s)': '-T-R---', 'Time': '1230-1420', 'Room': 'A122B', 'Instructor(s)': 'Katharine\xa0McEwen'}
{'Program Name': 'Anthropology', 'Course Code': 'ANTH 1120', 'CRN': '20293', 'Subject': 'ANTH', 'Course Number': '1120', 'Section': 'W01', 'Title': 'Intro\xa0to\xa0Cultural Anthropology', 'Type': 'WWW', 'Day(s)': '-------', 'Time': '-', 'Room': 'WWW', 'Instructor(s)': 'Adam\xa0Solomonian'}
{'Program Name': 'Anthropology', 'Course Code': 'ANTH 1221', 'CRN': '20736', 'Subject': 'ANTH', 'Course Number': '1221', 'Section': 'F01', 'Title': 'Field\xa0Techniques Archaeology', 'Type': 'Lecture', 'Day(s)': 'M------', 'Time': '1030-1220', 'Room': 'A235', 'Instructor(s)': 'Eric\xa0Simons'}
{'Program Name': 'Anthropology', 'Course Code': 'ANTH 1221', 'CRN': '', 'Subject': '', 'Course Number': '', 

In [75]:
df = pd.DataFrame(all_courses)

In [76]:
df.replace('', pd.NA, inplace=True)
df = df.fillna(method='ffill')

C:\Users\wudan\AppData\Local\Temp\ipykernel_8256\2550235393.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


In [74]:
df.iloc[4,4]

''

In [77]:
df.head(20)

,Program Name,Course Code,CRN,Subject,Course Number,Section,Title,Type,Day(s),Time,Room,Instructor(s)
0,Anthropology,ANTH 1120,20290,ANTH,1120,001,Intro to Cultural Anthropology,Lecture,-T-R---,1230-1420,A122B,Katharine McEwen
1,Anthropology,ANTH 1120,20293,ANTH,1120,W01,Intro to Cultural Anthropology,WWW,-------,-,WWW,Adam Solomonian
2,Anthropology,ANTH 1221,20736,ANTH,1221,F01,Field Techniques Archaeology,Lecture,M------,1030-1220,A235,Eric Simons
3,Anthropology,ANTH 1221,20736,ANTH,1221,F01,Field Techniques Archaeology,Field School,----F--,1030-1620,OFFC,Eric Simons
4,Anthropology,ANTH 1221,20736,ANTH,1221,F01,Field Techniques Archaeology,Lab,--W----,1030-1220,A235,Eric Simons
5,Anthropology,ANTH 1222,20737,ANTH,1222,F01,Fld Mthds Archaeology:Lab Anly,Lecture,M------,1430-1620,A235,Katharine McEwen
6,Anthropology,ANTH 1222,20737,ANTH,1222,F01,Fld Mthds Archaeology:Lab Anly,Lecture,--W----,1430-1620,A235,Katharine McEwen
7,Anthropology,ANTH 1223,20738,ANTH,1223,F01,Mthds in Archaelgy:Geosptl Ana,Lecture,M------,1230-1420,A235,Eric Simons
8,Anthropology,ANTH 1223,20738,ANTH,1223,F01,Mthds in Archaelgy:Geosptl Ana,Field School,----F--,1030-1620,OFFC,Eric Simons
9,Anthropology,ANTH 1223,20738,ANTH,1223,F01,Mthds in Archaelgy:Geosptl Ana,Lab,--W----,1230-1420,A235,Eric Simons


In [78]:
df.to_excel('..\courses.xlsx')

In [60]:
# Step 1: Split the HTML by Programs
program_blocks = soup.find_all('td', class_='deheader')

# Prepare a list to hold all courses across programs
all_courses = []
expected_details_length = 19  # Set this to the expected number of <td> elements in informative rows

# ignore_text = "Coursework for this section will be provided through Brightspace or other online platforms. Please contact the instructor or department for more information"

for program_block in program_blocks:
    program_name = program_block.find('span', class_='LargerText').get_text(strip=True)
    program_row = program_block.find_parent('tr')
    next_sib = program_row.find_next_sibling('tr')
    print(program_name)

    while next_sib and not next_sib.find('td', class_='deheader'):
        course_code_tag = next_sib.find('td', colspan="19", class_='dedefault')
        if course_code_tag:
            course_code = course_code_tag.get_text(strip=True)
            print(course_code)
            # Move to the next row, which should contain course details
            next_sib = next_sib.find_next_sibling('tr')

        while next_sib and not next_sib.find('td', colspan="19", class_='dedefault') and not next_sib.find('td', class_='deheader'):
            # Get the entire text of the row to check for the ignore text
            row_text = next_sib.get_text(strip=True)

            # Find the first 'td' with class 'dedefault' if it exists
            first_td = next_sib.find('td', class_='dedefault')

         # Proceed if the ignore text is not found in the row, and the first 'td' exists and does not contain a 'b' tag
            if ignore_text not in row_text and first_td and first_td.find('b') is None:
                details = [td.get_text(strip=True) for td in next_sib.select('td.dedefault, td.dedead')]
                print(details)
                if details:
                    course_details = {
                        'Program Name': program_name,
                        'Course Code': course_code,
                        'CRN': details[4],
                        'Subject': details[5],
                        'Course Number': details[6],
                        'Section': details[7],
                        'Title': details[9],
                        'Type': details[12],
                        'Day(s)': details[13],
                        'Time': details[14],
                        'Room': details[17],
                        'Instructor(s)': details[18]
                    }
                    all_courses.append(course_details)
            # Move to the next row
            next_sib = next_sib.find_next_sibling('tr')

# Print or process the extracted courses
for course in all_courses:
    print(course)




Anthropology
ANTH 1120
['', '66', '', '', '20290', 'ANTH', '1120', '001', '3.00', 'Intro\xa0to\xa0Cultural Anthropology', '', '-', 'Lecture', '-T-R---', '1230-1420', '', '', 'A122B', 'Katharine\xa0McEwen']
['', '23', '1', '', '20293', 'ANTH', '1120', 'W01', '3.00', 'Intro\xa0to\xa0Cultural Anthropology', '', '-', 'WWW', '-------', '-', '', '', 'WWW', 'Adam\xa0Solomonian']
ANTH 1221
['R', '20', '', '', '20736', 'ANTH', '1221', 'F01', '6.00', 'Field\xa0Techniques Archaeology', '', '-', 'Lecture', 'M------', '1030-1220', '06-May-24', '17-Aug-24', 'A235', 'Eric\xa0Simons']
['', '', '', '', '', '', '', '', '', '', '', '', 'Field School', '----F--', '1030-1620', '06-May-24', '17-Aug-24', 'OFFC', 'Eric\xa0Simons']
['', '', '', '', '', '', '', '', '', '', '', '', 'Lab', '--W----', '1030-1220', '06-May-24', '17-Aug-24', 'A235', 'Eric\xa0Simons']
ANTH 1222
['R', '20', '', '', '20737', 'ANTH', '1222', 'F01', '3.00', 'Fld\xa0Mthds\xa0Archaeology:Lab Anly', '', '-', 'Lecture', 'M------', '1430-1620

IndexError: list index out of range

In [43]:
# Step 1: Split the HTML by Programs
program_blocks = soup.find_all('td', class_='deheader')

# Prepare a list to hold all courses across programs
all_courses = []

for program_block in program_blocks:
    # Step 2: Parse Each Program Block
    program_name = program_block.find('span', class_='LargerText').get_text(strip=True)
    # Assuming each program's courses are enclosed within the same parent tag
    program_row = program_block.find_parent('tr')
    next_sib = program_row.find_next_sibling('tr')
    
    while next_sib and not next_sib.find('td', class_='deheader'):
        course_code_tag = next_sib.find('td', colspan="19", class_='dedefault')
        if course_code_tag:
            course_code = course_code_tag.get_text(strip=True)
            # Step 3: Extract Course Information
            # Find the row with course details
            course_details_row = next_sib.find_next_sibling('tr')
            details = [td.get_text(strip=True) for td in course_details_row.select('td.dedefault, td.dedead')]
            # print(details)
            # Extract course details based on the structure you provided
            course_details = {
                'Program Name': program_name,
                'Course Code': course_code,
                'CRN': details[4],
                'Subject': details[5],
                'Course Number': details[6],
                'Section': details[7],
                'Title': details[9],
                'Type': details[12],
                'Day(s)': details[13],
                'Time': details[14],
                'Room': details[17],
                'Instructor(s)': details[18]
            }
            all_courses.append(course_details)
        next_sib = next_sib.find_next_sibling('tr')

# Print or process the extracted courses
for course in all_courses:
    print(course)

{'Program Name': 'Anthropology', 'Course Code': 'ANTH 1120', 'CRN': '20290', 'Subject': 'ANTH', 'Course Number': '1120', 'Section': '001', 'Title': 'Intro\xa0to\xa0Cultural Anthropology', 'Type': 'Lecture', 'Day(s)': '-T-R---', 'Time': '1230-1420', 'Room': 'A122B', 'Instructor(s)': 'Katharine\xa0McEwen'}
{'Program Name': 'Anthropology', 'Course Code': 'ANTH 1221', 'CRN': '20736', 'Subject': 'ANTH', 'Course Number': '1221', 'Section': 'F01', 'Title': 'Field\xa0Techniques Archaeology', 'Type': 'Lecture', 'Day(s)': 'M------', 'Time': '1030-1220', 'Room': 'A235', 'Instructor(s)': 'Eric\xa0Simons'}
{'Program Name': 'Anthropology', 'Course Code': 'ANTH 1222', 'CRN': '20737', 'Subject': 'ANTH', 'Course Number': '1222', 'Section': 'F01', 'Title': 'Fld\xa0Mthds\xa0Archaeology:Lab Anly', 'Type': 'Lecture', 'Day(s)': 'M------', 'Time': '1430-1620', 'Room': 'A235', 'Instructor(s)': 'Katharine\xa0McEwen'}
{'Program Name': 'Anthropology', 'Course Code': 'ANTH 1223', 'CRN': '20738', 'Subject': 'ANTH'

In [52]:
l1 = ['RP', 'Seats Avail', '#\xa0on Waitlist', 'Sel', 'CRN', 'Subj', 'Crse', 'Sec', 'Cred', 'Title', "Add'l Fees", 'RptLimit', 'Type', 'Day(s)', 'Time', 'Non\xa0StandardStart/End', 'Room', 'Instructor(s)']
i = 0
for col in l1:
    print( i,col)
    i +=1

0 RP
1 Seats Avail
2 # on Waitlist
3 Sel
4 CRN
5 Subj
6 Crse
7 Sec
8 Cred
9 Title
10 Add'l Fees
11 RptLimit
12 Type
13 Day(s)
14 Time
15 Non StandardStart/End
16 Room
17 Instructor(s)


0 
1 66
2 
3 
4 20290
5 ANTH
6 1120
7 001
8 3.00
9 Intro to Cultural Anthropology
10 
11 -
12 Lecture
13 -T-R---
14 1230-1420
15 
16 
17 A122B
18 Katharine McEwen


In [18]:
program_blocks = soup.find_all('td', class_='deheader')

In [22]:
program_blocks[1].find('span', class_='LargerText').get_text(strip=True)

'Applied Planning'

In [23]:
program_blocks[1]

<td class="deheader" colspan="19" scope="col"><p class="centeraligntext"><a name="APPL"></a><span class="LargerText">Applied Planning</span></p></td>

In [30]:
 print(html_content)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<!-- saved from url=(0055)https://swing.langara.bc.ca/prod/bwckgens.P_RegsGetCrse -->
<html lang="en"><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">

<meta http-equiv="Pragma" name="Cache-Control" content="no-cache">
<meta http-equiv="Cache-Control" name="Cache-Control" content="no-cache">
<link rel="stylesheet" href="./Search for Courses__files/web_defaultapp.css" type="text/css">
<link rel="stylesheet" href="./Search for Courses__files/web_defaultprint.css" type="text/css" media="print">
<title>Search for Courses:</title>
<meta http-equiv="Content-Script-Type" name="Default_Script_Language" content="text/javascript">
<script language="JavaScript" type="text/javascript">
<!-- Hide JavaScript from older browsers 
window.onunload = function() {submitcount=0;}
var submitcount=0;
function checkSubmit() {
if (submitcount == 0)
   {
   submitcount++;
   return true;
   }
else
   {
alert("Your changes 